In [1]:
import pypeline
import numpy as np
from glob import glob
import os
import sys
import mne_bids
from contextlib import contextmanager
import mne
from datetime import datetime
%load_ext autoreload

%autoreload 2

In [ ]:
parent_dir = "../raw_data"  # directory where your raw data (folder containing brainvision, eyetracking asc, and behavior is stored)
data_dir = "../data"  # where to output data
if not os.path.exists(data_dir):
    os.makedirs(data_dir)


file_prefix = ""  # prefix to your vhdr files. Assuming it is in the format [prefix]_[number]

overwrite_subs = False  # if you want to overwrite the data for a subject, set to True
# TO PREPROCESS: 04 (session split across 2 recordings, which aren't concatenating well)
subject_dirs = ["18"]  # if you want to analyze a specific subset of subjects

if len(subject_dirs) == 0:
    subject_dirs = sorted(glob("*", root_dir=parent_dir))
if not overwrite_subs:
    subject_dirs = [sub for sub in subject_dirs if sub not in [f[4:] for f in glob("sub-*", root_dir=data_dir)]]

print("Subject to analyze: ", subject_dirs)


EXPERIMENT_NAME = "cassia"  # name of the experiment


TRIAL_START_TIME = -0.35  # epoch start before your designated timelock code
TRIAL_END_TIME = 2.7
BASELINE_TIME = (-0.3, 0)  # time for baseline correction
# NOTE: the WM trials are only 1200ms long, but tracking trials are 1,650ms before the delay, 215ms total

SRATE = 500  # hz, will resample if different from the raw data srate
FILTER_FREQS = (None, 60)  # None to not do one of the filtering steps

LINEAR_R2 = 0.3

event_dict = {  # this should be a dict of names of ALL the event codes that appear
    "trl_start": 1,
    "tracking/ss1": 21,
    "tracking/ss2": 22,
    "end_of_wm_first_pres": 2,
    "wm_second_pres": 3,
    "delay_start": 4,
    "delay_end": 5,
    "response": 6,
    "recording_start": 9,
    "data_end": 111,
}
for setsize in [1, 2]:
    for size in ["small", "large"]:
        for bin1 in range(6):
            for bin2 in range(6):
                final_code_piece = 0
                if setsize == 2:
                    final_code_piece = bin2 + 1  # we are shifting to 1 index so that 0 = absence
                elif setsize == 1 and size == "large":
                    final_code_piece = bin2 + 1  # that way we know what bins the large cloud spanned
                event_dict[f"WM/ss{setsize}/{bin1}/{final_code_piece}"] = (
                    (setsize - 1) * 100 + (bin1) * 10 + final_code_piece + 30
                )  # unique code for the trial, 30 shift keeps it separate from the rest


# event_dict is a list of name: number pairings for all the TRIAL event codes
# event_code_dict: a dict of code: sequence pairings for each trial
# so, if you have a trial with fixation (1) -> SS2 stimulus (12) -> delay (3) -> test (4), that you want to map to code  12:
# {12 : [1,12,3,4]}... and so on


track_conds = [21, 22]
event_code_dict = {
    21: [1, 21, 4, 5, 6],
    22: [1, 22, 4, 5, 6],
}
WM_conds = [v for k, v in event_dict.items() if "WM" in k]
for wmc in WM_conds:
    event_code_dict[wmc] = [1, wmc, 4, 5, 6]
stim_conditions = track_conds + WM_conds

POSITION_TO_TIMELOCK = 1  # which position (IN THE LIST ABOVE) to timelock to. Should be the one which indicates the condition. TODO: make this dynamic

EYEKEYWORD = "SYNC"

# edge case when we forgot to start the recording, manually drop certain trials
# Must be in the form of {'subject number':[list of ints]}
EEG_TRIALS_DROP = {
    "02": [352, 353, 354, 355, 356, 1446],  # trials when the eye tracker strangely dropped out
    "12": [805],
    "18": [992, 993, 994, 995],
}
EYE_TRIALS_DROP = {
    "16": [
        1559,
        1560,
        1561,
        1562,
        1563,
        1564,
    ],  # Trials after the EEG recording crashed, but the eye tracker kept running
    "18": list(np.arange(992, 1006)),
}
# channels to ignore when rejecting trials. Will also be excluded from analysis.
SUB_IGNORE_CHANS = {
    "07": ["Fp1", "Fp2"],
    "94": ["Fp1", "Fp2"],
}

NO_EYES = []  # subjects with no eye data
DROP_CHANNELS = (
    []
)  # channels to delete from the dataset entirely. Recommendation is to leave this blank and instead set REJ_TRIALS_IGNORE later on

Subject to analyze:  ['18']


In [4]:
pre = pypeline.Preprocess(
    data_dir=data_dir,
    root_dir=parent_dir,
    experiment_name=EXPERIMENT_NAME,
    srate=SRATE,
    trial_start=TRIAL_START_TIME,
    trial_end=TRIAL_END_TIME,
    event_dict=event_dict,
    event_code_dict=event_code_dict,
    timelock_ix=POSITION_TO_TIMELOCK,
    baseline_time=BASELINE_TIME,
    rejection_time=None,
    reject_between_codes=[1, 5],
    no_et_spaces=False,
    drop_channels=DROP_CHANNELS,
    filter_freqs=FILTER_FREQS,
)

In [4]:
@contextmanager
def redirect_stdout(new_stdout):  # writes the output to a log file
    save_stdout = sys.stdout
    save_stderr = sys.stderr
    sys.stdout = new_stdout
    sys.stderr = sys.stdout
    try:
        yield None
    finally:
        sys.stdout = save_stdout
        sys.stderr = save_stderr


# with open('preprocessing_log.txt','a+') as f:
#     with redirect_stdout(f):

print("\n\n\n##########################\n" + "STARTING PREPROCESSING RUN\n" + "##########################\n\n\n")
print(f'Run started at {datetime.now().strftime("%H:%M:%S")}')
for subject_number in subject_dirs:

    print(
        "\n\n#############################\n"
        + f"## STARTING NEW SUBJECT {subject_number} ##\n"
        + "#############################\n"
    )

    #####################
    #### IMPORT DATA ####
    #####################

    # import into the RAW bids dataset
    eeg, eeg_events = pre.import_eeg(subject_number, overwrite=True)
    reref_index = mne.pick_channels(eeg.ch_names, ["TP9"])  # TODO: custom rereferencing?
    eeg.load_data().apply_function(
        pre.rereference_to_average, picks=["eeg"], reref_values=np.squeeze(eeg.get_data()[reref_index])
    )
    eeg.filter(*pre.filter_freqs, n_jobs=-1)

    pre.import_behavior(subject_number)

    ########################################
    #### PREPROCESS EEG AND MAKE EPOCHS ####
    ########################################

    if subject_number in NO_EYES:
        epochs = pre.make_eeg_epochs(eeg, eeg_events, eeg_trials_drop=EEG_TRIALS_DROP.get(subject_number, None))
    else:
        eye, eye_events = pre.import_eyetracker(subject_number, keyword=EYEKEYWORD)

        epochs = pre.make_and_sync_epochs(
            eeg,
            eeg_events,
            eye,
            eye_events,
            eeg_trials_drop=EEG_TRIALS_DROP.get(subject_number, None),
            eye_trials_drop=EYE_TRIALS_DROP.get(subject_number, None),
        )

    ###############################
    #### DO ARTIFACT REJECTION ####
    ###############################

    p2p = pre.artreject_slidingP2P(
        epochs, rejection_criteria={"eeg": 100e-6, "eog": 200}, win=200, win_step=100
    )  # peak to peak in the window
    saccades = pre.artreject_step(
        epochs, rejection_criteria={"eyegaze": pre.deg2pix(0.5), "eog": 50}, win=80, win_step=10
    )  # saccades in EOG or eye tracking
    steps = pre.artreject_step(
        epochs, rejection_criteria={"eeg": 60e-6}, win=250, win_step=20
    )  # steps (saccade like) in EEG

    absolute_value = pre.artreject_value(
        epochs, rejection_criteria={"eyegaze": pre.deg2pix(1), "eeg": 100e-6, "eog": 300}
    )  # absolute value rejection
    linear_fit = pre.artreject_linear(epochs)  # linear fit (drift) rejection
    flatline = pre.artreject_flatline(
        epochs,
        rejection_criteria={"eeg": 0, "eog": 0, "eyegaze": 0},
        flatline_duration=200,
    )  # check for flatlines

    # combine rejection reasons
    rej_electrodes = p2p | saccades | steps | absolute_value | linear_fit | flatline
    rej_reasons = np.char.array(
        np.full(rej_electrodes.shape, "", dtype="<U30")
    )  # NOTE: dtype is important, must be >= the max possible str length
    rej_reasons[p2p] = "P2P "
    rej_reasons[saccades] = rej_reasons[saccades] + "SAC "
    rej_reasons[steps] = rej_reasons[steps] + "STEP "
    rej_reasons[absolute_value] = rej_reasons[absolute_value] + "ABS "
    rej_reasons[linear_fit] = rej_reasons[linear_fit] + "LIN "
    rej_reasons[flatline] = rej_reasons[flatline] + "FLAT "

    rej_counts = lambda x: f"{x.any(1).sum()} ({round(x.any(1).sum() / x.shape[0] * 100,1)}%)"
    print(
        (
            f"Rejected {rej_electrodes.any(1).sum()} trials ({round(rej_electrodes.any(1).sum() / rej_electrodes.shape[0] * 100,1)}%) for the following reasons:\n"
            f"Peak to peak amplitude: {rej_counts(p2p)}\n"
            f"Saccades: {rej_counts(saccades)}\n"
            f"Steps: {rej_counts(steps)}\n"
            f"Absolute value: {rej_counts(absolute_value)}\n"
            f"Linear fit: {rej_counts(linear_fit)}\n"
            f"Flatline: {rej_counts(flatline)}\n"
        )
    )

    print(
        "Worst electrodes by count:\n"
        + "\n".join(
            [f"{epochs.ch_names[i]}: {rej_electrodes[:,i].sum()}" for i in np.argsort(rej_electrodes.sum(0))[::-1][0:5]]
        )
    )

    #################################
    #### SAVE DATA AS DERIVATIVE ####
    #################################

    pre.save_all_data(subject_number, epochs, rej_reasons)
print(mne_bids.make_report(data_dir))




##########################
STARTING PREPROCESSING RUN
##########################



Run started at 10:15:09


#############################
## STARTING NEW SUBJECT 18 ##
#############################

More than 1 vhdr file present in subject directory. They will be concatenated in alphabetical order
Extracting parameters from ../raw_data/18/18.vhdr...
Setting channel info structure...
More than 1 vhdr file present in subject directory. They will be concatenated in alphabetical order
Extracting parameters from ../raw_data/18/19.vhdr...
Setting channel info structure...
More than 1 vhdr file present in subject directory. They will be concatenated in alphabetical order
Extracting parameters from ../raw_data/18/918.vhdr...
Setting channel info structure...


/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:175: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  mne.io.read_raw_brainvision(eegfile, eog=["HEOG", "VEOG"], misc=["StimTrak"], preload=False)
/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:175: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  mne.io.read_raw_brainvision(eegfile, eog=["HEOG", "VEOG"], misc=["StimTrak"], preload=False)
/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:175: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  mne.io.read_raw_brainvision(eegfile, eog=["HEOG", "VEOG"], misc=["StimTrak"], preload=False)
/Users/henryjones/Documents/research/cassi

Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S  4'), np.str_('Stimulus/S  5'), np.str_('Stimulus/S  6'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 30'), np.str_('Stimulus/S 32'), np.str_('Stimulus/S 40'), np.str_('Stimulus/S 43'), np.str_('Stimulus/S 50'), np.str_('Stimulus/S 54'), np.str_('Stimulus/S 60'), np.str_('Stimulus/S 65'), np.str_('Stimulus/S 70'), np.str_('Stimulus/S 76'), np.str_('Stimulus/S 80'), np.str_('Stimulus/S 81'), np.str_('Stimulus/S111'), np.str_('Stimulus/S132'), np.str_('Stimulus/S133'), np.str_('Stimulus/S134'), np.str_('Stimulus/S135'), np.str_('Stimulus/S136'), np.str_('Stimulus/S141'), np.str_('Stimulus/S143'), np.str_('Stimulus/S144'), np.str_('Stimulus/S145'), np.str_('Stimulus/S146'), np.str_('Stimulus/S151'), np.str_('Stimulus/S152'), np.str_('Stimulus/S154'), np.str_('Stimulus/S155'), np.str_('Stimulus/S156'), np.str_('Stimulus/S161'), np.str_('Stimulus/S162'), np.str_

/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:195: RuntimeWarning: Converting data files to BrainVision format
  mne_bids.write_raw_bids(
/opt/homebrew/Caskroom/miniforge/base/envs/preproc_pype/lib/python3.12/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


Writing '../data/sub-18/eeg/sub-18_task-cassia_eeg.json'...
Reading 0 ... 9147279  =      0.000 ...  9147.279 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 221 samples (0.221 s)



...

Rejected 35 trials (2.2%) for the following reasons:
Peak to peak amplitude: 29 (1.8%)
Saccades: 7 (0.4%)
Steps: 2 (0.1%)
Absolute value: 12 (0.7%)
Linear fit: 0 (0.0%)
Flatline: 0 (0.0%)

Worst electrodes by count:
F7: 28
FC5: 13
ypos_right: 6
ypos_left: 6
xpos_right: 3


/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:1006: RuntimeWarning: This filename (../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(path.fpath, overwrite=True)


Summarizing participants.tsv ../data/participants.tsv...
Summarizing scans.tsv files [PosixPath('../data/sub-13/sub-13_scans.tsv'), PosixPath('../data/sub-14/sub-14_scans.tsv'), PosixPath('../data/sub-15/sub-15_scans.tsv'), PosixPath('../data/sub-12/sub-12_scans.tsv'), PosixPath('../data/sub-08/sub-08_scans.tsv'), PosixPath('../data/sub-01/sub-01_scans.tsv'), PosixPath('../data/sub-06/sub-06_scans.tsv'), PosixPath('../data/sub-07/sub-07_scans.tsv'), PosixPath('../data/sub-09/sub-09_scans.tsv'), PosixPath('../data/sub-17/sub-17_scans.tsv'), PosixPath('../data/sub-10/sub-10_scans.tsv'), PosixPath('../data/sub-18/sub-18_scans.tsv'), PosixPath('../data/sub-11/sub-11_scans.tsv'), PosixPath('../data/sub-16/sub-16_scans.tsv'), PosixPath('../data/sub-05/sub-05_scans.tsv'), PosixPath('../data/sub-02/sub-02_scans.tsv'), PosixPath('../data/sub-94/sub-94_scans.tsv'), PosixPath('../data/sub-03/sub-03_scans.tsv'), PosixPath('../data/sub-04/sub-04_scans.tsv')]...


/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/preprocess.py:1008: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  np.save(path.fpath, epochs.get_data())


The participant template found: sex were all unknown;
handedness were all unknown;
ages all unknown
This dataset was created by [Unspecified] and conforms to BIDS version 1.7.0.
This report was generated with MNE-BIDS (https://doi.org/10.21105/joss.01896).
The dataset consists of 19 participants (sex were all unknown; handedness were
all unknown; ages all unknown) . Data was recorded using an EEG system (Brain
Products) sampled at 1000 Hz with line noise at 60 Hz. There were 19 scans in
total. Recording durations ranged from 1642.38 to 13126.4 seconds (mean =
9732.36, std = 2334.55), for a total of 184914.88 seconds of data recorded over
all scans. For each dataset, there were on average 34.0 (std = 0.0) recording
channels per scan, out of which 34.0 (std = 0.0) were used in analysis (0.0 +/-
0.0 were removed from analysis).


In [21]:
%matplotlib qt
sub = input('Enter subject number: ') # you can also just set this to a string

REJ_CHANNELS_IGNORE=['StimTrak', 'HEOG','VEOG','TP9'] # exclude fp1 and fp2 here if you want to keep trials where they are noisy, and drop them before analysis later


viz = pypeline.Visualizer(sub,
                            parent_dir = data_dir,
                            load_flags = False,  # we're going to rebuild our rejection flags every time
                            experiment_name=EXPERIMENT_NAME,
                            srate=SRATE,
                            rejection_time=[None, None],
                            downscale={'eyegaze':1e-6,'misc':1,'eeg':1,'eog':1e-6}, # convert to equivalent units (probably uV)
                            channels_drop=['pupil_left','pupil_right'],
                            channels_ignore=REJ_CHANNELS_IGNORE + SUB_IGNORE_CHANS.get(sub, []))


rejection_sums = viz.rej_chans.sum(axis=0)
sort_ix = np.argsort(rejection_sums)[::-1]

for ichan,chan in enumerate(viz.chan_labels[sort_ix]):
    if rejection_sums[sort_ix][ichan] > 0:
        print(chan,rejection_sums[sort_ix][ichan])
viz.preprocess_data_for_plot()
viz.open_figure()

Reading /Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_eeg.fif ...
    Found the data of interest:
        t =    -350.00 ...    2700.00 ms
        0 CTF compensation matrices available


/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/visualizer.py:57: RuntimeWarning: This filename (../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs_obj = mne.read_epochs(self.data_path.fpath)


Adding metadata with 84 columns
1604 matching events found
No baseline correction applied
0 projection items activated
F7 28
FC5 13
ypos_right 6
ypos_left 6
xpos_right 3
xpos_left 3
Fz 2
F3 2
CP5 2
FC1 2
C3 2
Fp1 2
Fp2 2
Pz 1
PO3 1
C4 1
Cz 1
CP2 1
F4 1
P7 1
P3 1
FC2 1
CP1 1
PO7 1


35/1604 trials rejected. Saving annotations as "../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_rejection_flags.npy"
35/1604 trials rejected. Saving annotations as "../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_rejection_flags.npy"
key not recognized: cmd. Press h for help.


In [ ]:
viz = pypeline.Visualizer(
    sub,
    parent_dir=data_dir,
    load_flags=False,  # we're going to rebuild our rejection flags every time
    experiment_name=EXPERIMENT_NAME,
    srate=SRATE,
    rejection_time=[None, None],
    downscale={"eyegaze": 1e-6, "misc": 1, "eeg": 1, "eog": 1e-6},  # convert to equivalent units (probably uV)
    channels_drop=["pupil_left", "pupil_right"],
    channels_ignore=REJ_CHANNELS_IGNORE + SUB_IGNORE_CHANS.get(sub, []),
)

Reading /Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_eeg.fif ...
    Found the data of interest:
        t =    -350.00 ...    2700.00 ms
        0 CTF compensation matrices available


/Users/henryjones/Documents/research/cassia/cassia4/preprocessing_pypeline/pypeline/visualizer.py:57: RuntimeWarning: This filename (../data/derivatives/sub-18/eeg/sub-18_task-cassia_desc-preprocessed_eeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs_obj = mne.read_epochs(self.data_path.fpath)


Adding metadata with 84 columns
1604 matching events found
No baseline correction applied
0 projection items activated
EVENTS_SHOW: 
['trl_start', 'tracking/ss1', 'tracking/ss2', 'end_of_wm_first_pres', 'wm_second_pres', 'delay_start', 'delay_end', 'response', 'recording_start', 'data_end', 'WM/ss1/0/0', 'WM/ss1/1/0', 'WM/ss1/2/0', 'WM/ss1/3/0', 'WM/ss1/4/0', 'WM/ss1/5/0', 'WM/ss1/0/1', 'WM/ss1/0/2', 'WM/ss1/0/3', 'WM/ss1/0/4', 'WM/ss1/0/5', 'WM/ss1/0/6', 'WM/ss1/1/1', 'WM/ss1/1/2', 'WM/ss1/1/3', 'WM/ss1/1/4', 'WM/ss1/1/5', 'WM/ss1/1/6', 'WM/ss1/2/1', 'WM/ss1/2/2', 'WM/ss1/2/3', 'WM/ss1/2/4', 'WM/ss1/2/5', 'WM/ss1/2/6', 'WM/ss1/3/1', 'WM/ss1/3/2', 'WM/ss1/3/3', 'WM/ss1/3/4', 'WM/ss1/3/5', 'WM/ss1/3/6', 'WM/ss1/4/1', 'WM/ss1/4/2', 'WM/ss1/4/3', 'WM/ss1/4/4', 'WM/ss1/4/5', 'WM/ss1/4/6', 'WM/ss1/5/1', 'WM/ss1/5/2', 'WM/ss1/5/3', 'WM/ss1/5/4', 'WM/ss1/5/5', 'WM/ss1/5/6', 'WM/ss2/0/1', 'WM/ss2/0/2', 'WM/ss2/0/3', 'WM/ss2/0/4', 'WM/ss2/0/5', 'WM/ss2/0/6', 'WM/ss2/1/1', 'WM/ss2/1/2', 'WM/ss2/